# Dependencies

In [39]:
import matplotlib.pyplot as plt
import os
import cv2 as cv
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from collections import Counter
import glob
from natsort import natsorted
from moviepy.editor import *

In [22]:
num2class = {"0.0" : "drone"}

# Bounding boxes

In [23]:
# box : (centerX, centerY, width, height)
def convertToAbsoluteValues(size, box):
    
    xIn = round(((2 * float(box[0]) - float(box[2])) * size[0] / 2))
    yIn = round(((2 * float(box[1]) - float(box[3])) * size[1] / 2))
    xEnd = xIn + round(float(box[2]) * size[0])
    yEnd = yIn + round(float(box[3]) * size[1])
    
    if xIn < 0:
        xIn = 0
    if yIn < 0:
        yIn = 0
    if xEnd >= size[0]:
        xEnd = size[0] - 1
    if yEnd >= size[1]:
        yEnd = size[1] - 1
    return (xIn, yIn, xEnd, yEnd)

# def convertToRelativeValues(size, box):
#     dw = 1. / (size[0])
#     dh = 1. / (size[1])
#     cx = (box[1] + box[0]) / 2.0
#     cy = (box[3] + box[2]) / 2.0
#     w = box[1] - box[0]
#     h = box[3] - box[2]
#     x = cx * dw
#     y = cy * dh
#     w = w * dw
#     h = h * dh
#     # x,y => (bounding_box_center)/width_of_the_image
#     # w => bounding_box_width / width_of_the_image
#     # h => bounding_box_height / height_of_the_image
#     return (x, y, w, h)

In [24]:
def boundingBoxes(labelPath, imagePath):
    
    detections, groundtruths, classes = [], [], []
    
    for boxtype in os.listdir(labelPath):

        boxtypeDir = os.path.join(labelPath,boxtype)
        
        for labelfile in os.listdir(boxtypeDir):
            filename = os.path.splitext(labelfile)[0]
            with open(os.path.join(boxtypeDir, labelfile)) as f: 
                labelinfos = f.readlines()   #텍스트 파일에 입력된 정보

            imgfilepath = os.path.join(imagePath, filename + ".jpg")
            img = cv.imread(imgfilepath)
            h, w, _ = img.shape

            for labelinfo in labelinfos:
                
                label, conf, rx1, ry1, rx2, ry2 = map(float, labelinfo.strip().split())
                x1, y1, x2, y2 = convertToAbsoluteValues((w, h), (rx1, ry1, rx2, ry2))
                boxinfo = [filename, label, conf, (x1, y1, x2, y2)]
                
                if label not in classes:
                    classes.append(label)
                
                if boxtype == "detection":
                    detections.append(boxinfo)
                else:
                    groundtruths.append(boxinfo)
                    
    classes = sorted(classes)
                
    return detections, groundtruths, classes
label_path = "C:/Users/qorwl/yolov5_py38/MAP_dic/label_kalman"
image_path = "C:/Users/qorwl/yolov5_py38/MAP_dic/image"

detections, groundtruths, classes = boundingBoxes(label_path, image_path)
print(detections)
print(groundtruths)
print(classes)

[['frame1', 0.0, 0.958083, (0, 0, 99, 18)], ['frame10', 0.0, 0.947169, (387, 96, 585, 177)], ['frame100', 0.0, 0.935155, (402, 189, 596, 264)], ['frame101', 0.0, 0.90602, (403, 191, 594, 262)], ['frame102', 0.0, 0.913095, (393, 191, 594, 263)], ['frame103', 0.0, 0.792977, (393, 191, 597, 263)], ['frame104', 0.0, 0.704584, (397, 191, 597, 263)], ['frame105', 0.0, 0.725369, (402, 193, 597, 262)], ['frame106', 0.0, 0.725369, (408, 195, 603, 263)], ['frame107', 0.0, 0.794139, (399, 192, 598, 270)], ['frame108', 0.0, 0.794139, (407, 192, 605, 270)], ['frame109', 0.0, 0.794139, (407, 193, 606, 271)], ['frame11', 0.0, 0.954187, (387, 96, 585, 176)], ['frame110', 0.0, 0.794139, (408, 194, 606, 272)], ['frame111', 0.0, 0.794139, (408, 195, 607, 273)], ['frame112', 0.0, 0.794139, (409, 196, 607, 274)], ['frame113', 0.0, 0.794139, (409, 197, 607, 275)], ['frame114', 0.0, 0.794139, (409, 198, 608, 276)], ['frame115', 0.0, 0.794139, (410, 199, 608, 277)], ['frame116', 0.0, 0.794139, (410, 200, 609,

# Plot Bounding boxes

In [1]:
def boxPlot(boxlist, imagePath, savePath):
    labelfiles = sorted(list(set([filename for filename, _, _, _ in boxlist])))
    
    for labelfile in labelfiles:
    
        rectinfos = []
        imgfilePath = os.path.join(imagePath, labelfile + ".jpg")
        img = cv.imread(imgfilePath)

        for filename, _, conf, (x1, y1, x2, y2) in boxlist:
            if labelfile == filename:
                rectinfos.append((x1, y1, x2, y2, conf))
                
        for x1, y1, x2, y2, conf in rectinfos:
            
            if conf == 1.0:
                rectcolor = (0, 255, 0)
            else:
                rectcolor = (0, 0, 255)
                
            cv.rectangle(img, (x1, y1), (x2, y2), rectcolor, 4)
        cv.imwrite(f"{savePath}/{labelfile}.jpg", img)

        img = mpimg.imread(f"{savePath}/{labelfile}.jpg")
        plt.axis("off")
        plt.imshow(img)
        plt.show()
        
# boxPlot(detections, "image", savePath="boxed_images/detection")
# boxPlot(groundtruths, "image", savePath="boxed_images/groundtruth")
savePath="boxed_images/both_kalman"
boxPlot(detections + groundtruths, "image", savePath)

NameError: name 'detections' is not defined

# Save AVI File

In [26]:
img_array = []

for filename in natsorted(glob.glob(savePath+'/*.jpg')):
    
    img = cv.imread(filename)
    
    h, w, layers = img.shape
    size = ( w , h )
    img_array.append(img)
        
out = cv.VideoWriter('Kalman_Result.avi', cv.VideoWriter_fourcc(*'DIVX'), 30,  size)
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()
    


# Save GIF File

In [40]:
VideoFileClip('Kalman_Result.avi').speedx(4).write_gif('Kalman_Result.gif')

MoviePy - Building file Kalman_Result.gif with imageio.


# IoU(Intersection over Union)

In [27]:
def getArea(box):
    return (box[2] - box[0] + 1) * (box[3] - box[1] + 1)


def getUnionAreas(boxA, boxB, interArea=None):
    area_A = getArea(boxA)
    area_B = getArea(boxB)
    
    if interArea is None:
        interArea = getIntersectionArea(boxA, boxB)
        
    return float(area_A + area_B - interArea)

def getIntersectionArea(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    # intersection area
    return (xB - xA + 1) * (yB - yA + 1)

# boxA = (Ax1,Ay1,Ax2,Ay2)
# boxB = (Bx1,By1,Bx2,By2)
def boxesIntersect(boxA, boxB):
    if boxA[0] > boxB[2]:
        return False  # boxA is right of boxB
    if boxB[0] > boxA[2]:
        return False  # boxA is left of boxB
    if boxA[3] < boxB[1]:
        return False  # boxA is above boxB
    if boxA[1] > boxB[3]:
        return False  # boxA is below boxB
    return True

def iou(boxA, boxB):
    # if boxes dont intersect
    if boxesIntersect(boxA, boxB) is False:
        return 0
    interArea = getIntersectionArea(boxA, boxB)
    union = getUnionAreas(boxA, boxB, interArea=interArea)
    
    # intersection over union
    result = interArea / union
    assert result >= 0
    return result

In [28]:
boxA = detections[-1][-1]
boxB = groundtruths[-1][-1]

print(f"boxA coordinates : {(boxA)}")
print(f"boxA area : {getArea(boxA)}")
print(f"boxB coordinates : {(boxB)}")
print(f"boxB area : {getArea(boxB)}")

print(f"Union area of boxA and boxB : {getUnionAreas(boxA, boxB)}")

print(f"Does boxes Intersect? : {boxesIntersect(boxA, boxB)}")

print(f"Intersection area of boxA and boxB : {getIntersectionArea(boxA, boxB)}")

print(f"IoU of boxA and boxB : {iou(boxA, boxB)}")

boxA coordinates : (405, 186, 596, 263)
boxA area : 14976
boxB coordinates : (400, 198, 603, 267)
boxB area : 14280
Union area of boxA and boxB : 16584.0
Does boxes Intersect? : True
Intersection area of boxA and boxB : 12672
IoU of boxA and boxB : 0.76410998552822


# AP(Average Precision)

In [29]:
def calculateAveragePrecision(rec, prec):
    
    mrec = [0] + [e for e in rec] + [1]
    mpre = [0] + [e for e in prec] + [0]

    for i in range(len(mpre)-1, 0, -1):
        mpre[i-1] = max(mpre[i-1], mpre[i])

    ii = []

    for i in range(len(mrec)-1):
        if mrec[1:][i] != mrec[0:-1][i]:
            ii.append(i+1)

    ap = 0
    for i in ii:
        ap = ap + np.sum((mrec[i] - mrec[i-1]) * mpre[i])
    
    return [ap, mpre[0:len(mpre)-1], mrec[0:len(mpre)-1], ii]

In [30]:
def ElevenPointInterpolatedAP(rec, prec):

    mrec = [e for e in rec]
    mpre = [e for e in prec]

    recallValues = np.linspace(0, 1, 11)
    recallValues = list(recallValues[::-1])
    rhoInterp, recallValid = [], []

    for r in recallValues:
        argGreaterRecalls = np.argwhere(mrec[:] >= r)
        pmax = 0

        if argGreaterRecalls.size != 0:
            pmax = max(mpre[argGreaterRecalls.min():])

        recallValid.append(r)
        rhoInterp.append(pmax)

    ap = sum(rhoInterp) / 11

    return [ap, rhoInterp, recallValues, None]

In [34]:
def AP(detections, groundtruths, classes, IOUThreshold = 0.4, method = 'AP'):
    
    result = []
    
    for c in classes:

        dects = [d for d in detections if d[1] == c]
        gts = [g for g in groundtruths if g[1] == c]

        npos = len(gts)

        dects = sorted(dects, key = lambda conf : conf[2], reverse=True)

        TP = np.zeros(len(dects))
        FP = np.zeros(len(dects))

        det = Counter(cc[0] for cc in gts)

        # 각 이미지별 ground truth box의 수
        # {99 : 2, 380 : 4, ....}
        # {99 : [0, 0], 380 : [0, 0, 0, 0], ...}
        for key, val in det.items():
            det[key] = np.zeros(val)


        for d in range(len(dects)):


            gt = [gt for gt in gts if gt[0] == dects[d][0]]

            iouMax = 0

            for j in range(len(gt)):
                iou1 = iou(dects[d][3], gt[j][3])
                if iou1 > iouMax:
                    iouMax = iou1
                    jmax = j

            if iouMax >= IOUThreshold:
                if det[dects[d][0]][jmax] == 0:
                    TP[d] = 1
                    det[dects[d][0]][jmax] = 1
                else:
                    FP[d] = 1
            else:
                FP[d] = 1

        acc_FP = np.cumsum(FP)
        acc_TP = np.cumsum(TP)
        rec = acc_TP / npos
        prec = np.divide(acc_TP, (acc_FP + acc_TP))

        if method == "AP":
            [ap, mpre, mrec, ii] = calculateAveragePrecision(rec, prec)
        else:
            [ap, mpre, mrec, _] = ElevenPointInterpolatedAP(rec, prec)

        r = {
            'class' : c,
            'precision' : prec,
            'recall' : rec,
            'AP' : ap,
            'interpolated precision' : mpre,
            'interpolated recall' : mrec,
            'total positives' : npos,
            'total TP' : np.sum(TP),
            'total FP' : np.sum(FP)
        }

        result.append(r)

    return result

result = AP(detections, groundtruths, classes)
print(result)

[{'class': 0.0, 'precision': array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.        , 1.        , 1.        , 1.        , 0.9       ,
       0.90909091, 0.91666667, 0.92307692, 0.92857143, 0.93333333,
       0.9375    , 0.94117647, 0.94444444, 0.94736842, 0.95      ,
       0.95238095, 0.95454545, 0.95652174, 0.95833333, 0.96      ,
       0.96153846, 0.96296296, 0.96428571, 0.96551724, 0.96666667,
       0.96774194, 0.96875   , 0.96969697, 0.97058824, 0.97142857,
       0.97222222, 0.97297297, 0.97368421, 0.97435897, 0.975     ,
       0.97560976, 0.97619048, 0.97674419, 0.97727273, 0.97777778,
       0.97826087, 0.9787234 , 0.97916667, 0.97959184, 0.98      ,
       0.98039216, 0.98076923, 0.98113208, 0.98148148, 0.98181818,
       0.98214286, 0.98245614, 0.98275862, 0.98305085, 0.98333333,
       0.98360656, 0.98387097, 0.98412698, 0.984375  , 0.98461538,
       0.98484848, 0.98507463, 0.98529412, 0.98550725, 0.98571429,
       0.98591549, 0.98611111, 0.

# mAP(mean Average Precision)

In [35]:
def mAP(result):
    ap = 0
    for r in result:
        ap += r['AP']
    mAP = ap / len(result)
    
    return mAP

In [36]:
for r in result:
    print("{:^8} AP : {}".format(num2class[str(r['class'])], r['AP']))
print("---------------------------")
print(f"mAP : {mAP(result)}")

 drone   AP : 0.6818796637877651
---------------------------
mAP : 0.6818796637877651
